In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer

In [2]:
mi_df =  pd.read_csv(
    'data/01_data_cruda/MI_data.csv', 
    header=0, 
    sep=","
)

There  are  four  possible  time  moments  for  complication  prediction:  on  base  of  the information known at

1.the time of admission to hospital:all input columns (2-112) except 93, 94, 95, 100, 101, 102, 103, 104, 105can be usedfor prediction;

2.the end of the first day(24 hours after admission to the hospital):all input columns (2-112) except 94, 95, 101, 102, 104, 105 can be usedfor prediction;

3.the end of the second day (48 hours after admission to the hospital) all input columns (2-112) except 95, 102, 105 can be usedfor prediction;

4.the end of the third day (72 hours after admission to the hospital) all input columns (2-112) can be usedfor prediction.

---




In [3]:
mi_df.iloc[:,112:124].columns.tolist()

['FIBR_PREDS',
 'PREDS_TAH',
 'JELUD_TAH',
 'FIBR_JELUD',
 'A_V_BLOK',
 'OTEK_LANC',
 'RAZRIV',
 'DRESSLER',
 'ZSN',
 'REC_IM',
 'P_IM_STEN',
 'LET_IS']

In [4]:
# Hacemos referencia a las columnas 2-112 (Variables que pueden ser usadas para la predicción)

# Referenciamos columnas con indice de 0 a 111
var_objetivo_lst = [
    'FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD',
    'A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN',
    'REC_IM','P_IM_STEN'
    #,'LET_IS'
]
mi_objetivo_df = mi_df[var_objetivo_lst]
mi_predictoras_df = mi_df.iloc[:,0:112]

mi_df = pd.concat([mi_predictoras_df, mi_objetivo_df], axis=1, join='inner')
# Removemos columnas 93, 94 , 95, 101, 102, 103, 104, 105
mi_df.drop(mi_df.columns[[1-1, 93-1, 94-1 , 95-1, 101-1, 102-1, 103-1, 104-1, 105-1]],axis = 1, inplace=True)

### 1. Removemos columnas

#### 1.1 Remover columnas que tienen un porcentaje de valores únicos superior al 70%

In [5]:
#Determinamos aquellas columnas (variables) que tengan un porcentaje de filas (registros) únicas por cada columna (variable) mayor al valor de 0.7
threshold = 0.7

unique_percentages = mi_df.nunique() / len(mi_df)

criteria = unique_percentages > threshold

columns_to_filter = unique_percentages[criteria].keys()

#Eliminamos las columnas (variables) seleccionadas en el paso anterior. Al aplicar "drop" obtenemos un nuevo "DataFrame", pero con el parámetro
#inplace igual a "True" podemos realizar dicha operación de eliminación y sobre-escribir el "DataFrame" del cual estamos eliminando con el
#nuevo "DataFrame" obtenido. El parámetro axis permite seleccionar el eje respecto al cual aplicar la operación "drop". Los valores válidos
#son [0 o "index"] y [1 o "columns"]
mi_df.drop(columns_to_filter, axis=1, inplace=True)

### 2. Tratamiento de datos perdidos

In [6]:
from collections import defaultdict
from sklearn import preprocessing 

In [7]:
### El dataset solo contiene variables numéricas y no contiene variables categóricas que tengan que ser codificadas

# Agrupando columnas por tipo de datos
tipos = mi_df.columns.to_series().groupby(mi_df.dtypes).groups

# Armando lista de columnas categóricas
cint = tipos[np.dtype('int64')]
# Armando lista de columnas numéricas
cfloat = tipos[np.dtype("float64")]

In [8]:
## Para columnas que tengan > 20% de valores perdidos: Eliminar
cols_mayor_a_20 = [col for col in mi_df.columns if ((mi_df[col].isnull().sum() / len(mi_df)*100) > 20.0)]

mi_df.drop(
    cols_mayor_a_20, 
    axis = 1, 
    inplace=True
)

In [9]:
## Para columnas que tengan entre 15% y 20% de valores perdidos: Imputar con modelos
cols_entre_15_y_20 = [
    col for col in mi_df.columns if 
        (((mi_df[col].isnull().sum() / len(mi_df)*100))>=15 and  
        ((mi_df[col].isnull().sum() / len(mi_df)*100))<=20)
]

In [11]:
#sns.pairplot(mi_df)

mi_imp_wt_model_df = mi_df.copy() 
mi_imp_wt_model_df.drop(
    var_objetivo_lst,
    axis = 1, 
    inplace=True
)

es_null=pd.isna(mi_imp_wt_model_df.loc[:, 'S_AD_ORIT'])
#Selecciona solo los que contiene "True"
df_m_null = mi_imp_wt_model_df.loc[es_null]
df_m_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 32 to 1581
Data columns (total 92 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AGE            265 non-null    float64
 1   SEX            267 non-null    int64  
 2   INF_ANAM       267 non-null    float64
 3   STENOK_AN      258 non-null    float64
 4   FK_STENOK      259 non-null    float64
 5   IBS_POST       263 non-null    float64
 6   GB             267 non-null    float64
 7   SIM_GIPERT     267 non-null    float64
 8   DLIT_AG        244 non-null    float64
 9   ZSN_A          266 non-null    float64
 10  nr_11          266 non-null    float64
 11  nr_01          266 non-null    float64
 12  nr_02          266 non-null    float64
 13  nr_03          266 non-null    float64
 14  nr_04          266 non-null    float64
 15  nr_07          266 non-null    float64
 16  nr_08          266 non-null    float64
 17  np_01          266 non-null    float64
 18  np_04   

In [13]:
mi_df["AST_BLOOD"].unique()

array([ nan, 0.18, 0.11, 0.37, 0.22, 0.45, 0.6 , 0.15, 0.07, 0.38, 0.3 ,
       0.26, 0.44, 0.41, 0.52, 0.5 , 0.33, 0.04, 0.67, 0.75, 0.2 , 1.34,
       0.4 , 1.12, 0.86, 0.48, 0.96, 1.04, 0.63, 0.14, 0.13, 0.08, 1.2 ,
       0.36, 0.9 , 0.82, 0.56, 0.12, 0.57, 0.66, 1.13, 2.15, 0.27, 0.8 ,
       0.46, 0.74, 0.23, 0.1 , 0.7 , 1.05, 0.28, 0.68, 0.98, 1.36, 0.71,
       1.75, 0.97, 1.08, 0.49])

In [14]:
mi_df[cols_entre_15_y_20]

,S_AD_ORIT,D_AD_ORIT,ALT_BLOOD,AST_BLOOD
0,180.0,100.0,NaN,NaN
1,120.0,90.0,0.38,0.18
2,180.0,100.0,0.30,0.11
3,120.0,70.0,0.75,0.37
4,160.0,90.0,0.45,0.22
...,...,...,...,...
1695,110.0,70.0,1.05,0.52
1696,50.0,0.0,NaN,NaN
1697,70.0,50.0,0.23,0.15
1698,110.0,70.0,0.45,0.45


In [15]:
## Para columnas que tengan menos de 15% de valores perdidos: Imputar con Media, Moda y Mediana
cols_menor_a_15 = [col for col in mi_df.columns if ((mi_df[col].isnull().sum() / len(mi_df)*100) < 15.0)]

Notas de la libreria SimpleImputer para el parametro "strategy"

- Si es "mean", reemplace los valores faltantes usando la media a lo largo de cada columna. Solo se puede usar con datos numéricos flotantes (float).

- Si es "median", reemplace los valores faltantes usando la mediana a lo largo de cada columna. Solo se puede usar con datos numéricos enteros (int).

- Si es "most_frequent", reemplace falta utilizando el valor más frecuente a lo largo de cada columna. Se puede usar con cadenas o datos numéricos que representan variables categóricas (str/int).

In [16]:
imp_media = SimpleImputer(missing_values=np.nan, #etiqueta de valores nulos
                    strategy='mean') #método o estrategia de imputación
#imp_media es la función con los parámetros ya especificados de nuestra función

imp_moda = SimpleImputer(missing_values=np.nan, #etiqueta de valores nulos
                    strategy='most_frequent') #método o estrategia de imputación
#imp_moda es la función con los parámetros ya especificados de nuestra función

In [18]:
missing_cols_int_lst = [col for col in cols_menor_a_15 if col in cint]

mi_int_limpio_df = imp_moda.fit_transform(mi_df[missing_cols_int_lst])
mi_int_limpio_df = pd.DataFrame(mi_int_limpio_df, columns=missing_cols_int_lst)

In [19]:
missing_cols_float_lst = [col for col in cols_menor_a_15 if col in cfloat]

mi_float_limpio_df = imp_media.fit_transform(mi_df[missing_cols_float_lst])
mi_float_limpio_df = pd.DataFrame(mi_float_limpio_df, columns=missing_cols_float_lst)

In [20]:
mi_df.drop(
    missing_cols_int_lst+missing_cols_float_lst,
    axis = 1, 
    inplace=True
)
mi_df = pd.concat([mi_df, mi_int_limpio_df], axis=1, join='inner')
mi_df = pd.concat([mi_df, mi_float_limpio_df], axis=1, join='inner')

In [22]:
mi_df["ZSN"].unique()

array([0, 1], dtype=int64)

In [23]:
((mi_df.isnull().sum() / len(mi_df)*100)>0.001).sum()

4

In [24]:
mi_df["FIBR_PREDS"]

0       0
1       0
2       0
3       0
4       0
       ..
1695    0
1696    0
1697    0
1698    0
1699    0
Name: FIBR_PREDS, Length: 1700, dtype: int64

In [25]:
mi_df

,S_AD_ORIT,D_AD_ORIT,ALT_BLOOD,AST_BLOOD,SEX,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,...,TIME_B_S,NITR_S,NA_R_1_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n
0,180.0,100.0,NaN,NaN,1,0,0,0,0,0,...,4.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,120.0,90.0,0.38,0.18,1,0,0,0,0,0,...,2.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
2,180.0,100.0,0.30,0.11,1,0,0,0,0,0,...,3.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3,120.0,70.0,0.75,0.37,0,0,0,0,0,0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
4,160.0,90.0,0.45,0.22,1,0,0,0,0,0,...,9.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,110.0,70.0,1.05,0.52,0,0,0,0,0,1,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1696,50.0,0.0,NaN,NaN,0,0,0,0,0,0,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1697,70.0,50.0,0.23,0.15,1,0,0,0,0,0,...,1.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
1698,110.0,70.0,0.45,0.45,0,0,0,0,0,0,...,2.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0


In [26]:
mi_df["COMPLICATION_OUT"] = mi_df[var_objetivo_lst].apply(
    lambda x: x["FIBR_PREDS"]+
              x["PREDS_TAH"]+
              x["JELUD_TAH"],
    axis=1
)


# var_objetivo_lst = [
#     'FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD',
#     'A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN',
#     'REC_IM','P_IM_STEN'
#     #,'LET_IS'
# ]

In [27]:
mi_df["COMPLICATION_OUT"]

0       0
1       0
2       0
3       0
4       0
       ..
1695    0
1696    0
1697    0
1698    0
1699    0
Name: COMPLICATION_OUT, Length: 1700, dtype: int64

In [28]:
mi_df.to_csv('data/02_data_intermedia/MI_preprocesado.csv')